# 5. 蒙特卡洛方法

## 5.1 蒙特卡洛方法的基本框架

蒙特卡洛方法（Monte Carlo Methods）最早由 Boyle (1977) 应用于期权定价。如今，它已广泛用于定价具有复杂特性的金融衍生品。

蒙特卡洛方法的核心思想是利用**随机抽样**来逼近期望值。设我们希望估计某个参数 $\theta$，其数学期望可表示为：

$$
\theta = \mathbb{E}[g(X)]
$$

其中，$g(X)$ 是某个函数，满足 $\mathbb{E}[|g(X)|] < \infty$。我们可以生成 $n$ 个独立随机变量 $X_1, X_2, \dots, X_n$，其分布由概率密度函数 $f(X)$ 给出。

于是，$\theta$ 的蒙特卡洛估计值为：

$$
\hat{\theta} = \frac{1}{n} \sum_{i=1}^{n} g(X_i)
$$

根据**大数定律**（Law of Large Numbers, LLN），当 $n \to \infty$ 时：

$$
\hat{\theta} \to \theta \quad \text{（几乎必然收敛）}
$$

然而，蒙特卡洛方法的结果并非完全准确，因为其估计值存在**随机误差**。我们需要考虑其方差：

$$
s^2 = \frac{1}{n - 1} \sum_{i=1}^{n} (g(X_i) - \hat{\theta})^2
$$

根据**中心极限定理**（Central Limit Theorem, CLT），当 $n \to \infty$ 时：

$$
\sqrt{n} \frac{(\hat{\theta} - \theta)}{s} \sim N(0,1)
$$

这意味着我们可以用**置信区间**来衡量估计值的可靠性。例如，在 95% 置信水平下：

$$
\left( \hat{\theta} - \frac{1.96 s}{\sqrt{n}}, \hat{\theta} + \frac{1.96 s}{\sqrt{n}} \right)
$$

提供了对真实值 $\theta$ 的估计范围。

不过，蒙特卡洛方法的**收敛速度较慢**。由于标准误差的下降速率是 $O(1/\sqrt{n})$，若希望将误差减少至原来的 1/10，则需要增加 100 倍的样本数。因此，提高计算效率的关键在于**降低方差**，而不仅仅是增加样本数量。

---

## 5.2 方差减少技术（Variance Reduction Techniques）

蒙特卡洛方法的主要缺点是**收敛速度慢**，其误差下降速率为：

$$
O\left(\frac{1}{\sqrt{n}}\right)
$$

这意味着如果我们希望将误差减少 10 倍，则需要增加 100 倍的样本量，计算成本巨大。因此，我们希望在**不增加样本数量**的情况下，降低方差，提高收敛效率。

方差减少技术的核心思路是**调整模拟方法，使得估计量的方差变小**，从而提高精度。下面详细介绍两种重要的方差减少技术：

---

### 5.2.1 控制变量法（Control Variates）

#### 1. 基本思想

控制变量法的核心是找到一个**已知期望值**的随机变量，并利用它来调整估计值，从而减少方差。

假设我们希望估计：

$$
\theta = \mathbb{E}[Y], \quad Y = g(X)
$$

而我们找到另一个随机变量 $Z$，其期望值已知：

$$
\mathbb{E}[Z] = \theta_Z
$$

我们可以构造一个新的估计量：

$$
\hat{\theta}_c = Y + c(Z - \theta_Z)
$$

其中，$c$ 是一个待优化的常数。

#### 2. 为什么这样做可以降低方差？

计算新的估计量的方差：

$$
\text{Var}(\hat{\theta}_c) = \text{Var}(Y) + c^2 \text{Var}(Z) - 2c \text{Cov}(Y, Z)
$$

我们希望通过选择最优的 $c$ 来最小化方差，对 $c$ 求导并令其为 0：

$$
\frac{d}{dc} \text{Var}(\hat{\theta}_c) = 2c \text{Var}(Z) - 2 \text{Cov}(Y, Z) = 0
$$

解得：

$$
c^* = -\frac{\text{Cov}(Y, Z)}{\text{Var}(Z)}
$$

代入后，方差最小化：

$$
\text{Var}(\hat{\theta}_c) = \text{Var}(Y) - \frac{\text{Cov}^2(Y, Z)}{\text{Var}(Z)}
$$

由于协方差不可能超过两个变量方差的乘积，即：

$$
\frac{\text{Cov}^2(Y, Z)}{\text{Var}(Z)} \leq \text{Var}(Y)
$$

所以只要 $\text{Cov}(Y, Z) \neq 0$，新估计量的方差必然小于原估计量的方差，说明该方法确实**有效降低了方差**。

---

### 5.2.2 对偶变量法（Antithetic Variates）

#### 1. 基本思想

对偶变量法的核心是**利用负相关样本对来减少方差**。

假设我们希望估计：

$$
\theta = \mathbb{E}[g(X)]
$$

而我们可以生成成对的样本 $(X_i, -X_i)$，并构造新的估计量：

$$
\hat{\theta}_a = \frac{1}{n} \sum_{i=1}^{n} \frac{g(X_i) + g(-X_i)}{2}
$$

计算方差：

$$
\text{Var}(\hat{\theta}_a) = \frac{1}{4} \left( \text{Var}(g(X)) + \text{Var}(g(-X)) + 2\text{Cov}(g(X), g(-X)) \right)
$$

如果 $g(X)$ 是**单调函数**，则 $g(X)$ 与 $g(-X)$ **负相关**（$\text{Cov}(g(X), g(-X)) < 0$），使方差减少，提高计算效率。

---

### 5.2.3 其他方差减少技术

除了**控制变量法**和**对偶变量法**，还有其他常见的方法，后续再做拓展：

1. **重要性抽样（Importance Sampling）**：
   - 通过改变随机变量的分布，使得对最终结果贡献较大的样本被更多地采样，从而减少方差。

2. **分层抽样（Stratified Sampling）**：
   - 将样本空间划分为多个子区间，在每个子区间内均匀抽样，从而减少随机性误差。
